# MCP tool calling with the WRITER Node SDK

This notebook demonstrates how to use the **WRITER MCP (Model Control Protocol) Node SDK** to connect AI agents to WRITER’s ecosystem of tools — including file handling, knowledge graph operations, and content generation — through a standardized protocol layer.

---

## About the tools in WRITER’s MCP package

Below are some of the core tools exposed by the WRITER MCP environment:

| Tool | Purpose |
|------|----------|
| `file_read` | Read the contents of a file (local or from resources). |
| `upload_files` | Upload file bytes or text content to the WRITER platform. |
| `create_graphs` | Create a new knowledge graph instance. |
| `add_file_to_graph_graphs` | Associate an uploaded file with an existing knowledge graph. |
| `question_graphs` | Query or ask questions against data in a knowledge graph. |
| *(Other tools)* | Such as `update_graphs`, `delete_file`, `generate_content_applications`, etc. |

These tools are the building blocks that let an AI agent **ingest**, **organize**, and **reason over structured and unstructured data** inside the WRITER environment.

---

## Source and documentation

You can find the MCP server implementation and reference details in WRITER’s official repository:

📦 [**WRITER MCP server — GitHub**](https://github.com/writer/writer-node/tree/main/packages/mcp-server)

That package contains the server-side tool definitions and schemasfor integrating MCP endpoints into WRITER’s platform.


## Prerequisites
- A [WRITER AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)
- Basic familiarity with Python and [AWS Strands](https://dev.writer.com/home/integrations/strands)

## Installation and setup

First, we need to install the WRITER SDK MCP package globally using npm. This provides the MCP server that will allow us to interact with WRITER's AI tools and services.


In [ ]:
!npm install -g writer-sdk-mcp

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os

os.environ["WRITER_API_KEY"] = getpass.getpass()


## Starting the MCP server

Now we'll start the WRITER MCP server on port 3000. This server acts as a bridge between our Python code and WRITER's services, providing access to tools like content generation, knowledge graphs, file processing, and more. The server runs in the background using subprocess.


In [ ]:
import subprocess

# Start MCP server on port 3000 (can change if needed)
server = subprocess.Popen(
    ["npx", "-y", "writer-sdk-mcp", "--transport=http", "--port=3000"],
    env=os.environ,
)
print("✅ WRITER MCP Server running at http://localhost:3000")


Let's test our connection to the MCP server by sending and initilization request. This uses the **JSON-RPC 2.0** protocol to establish communication with the **WRITER MCP server** running at `http://localhost:3000`.

In [ ]:
import requests
import json

base_url = "http://localhost:3000"

headers = {
    "Accept": "application/json, text/event-stream",
    "Content-Type": "application/json"
}

payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
      "protocolVersion": "1.0",
      "capabilities": {},
      "clientInfo": {"name": "test-client", "version": "0.1"}
    }
}



with requests.post(base_url, json=payload, headers=headers, stream=True) as resp:
    print("Status:", resp.status_code)
    print("Content-Type:", resp.headers.get("Content-Type"))
    print("\n🔹 Streaming MCP events:\n")

    for line in resp.iter_lines(decode_unicode=True):
        if line.strip():
            print(line)


## Example: building an assistant with WRITER + Strands Agents SDK

For this example, we’ll create a simple assistant using the [**WRITER Strands Agents SDK integration**](https://dev.writer.com/home/integrations/strands), which connects the **WRITER MCP SDK** (Node package) to the **Strands Agents** framework.  
This setup allows our agents to call WRITER’s MCP tools directly through the [**Strands `MCPClient`**](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/mcp-tools/) interface.


### Python dependencies
We'll install the **Strands Agents framework** with the **WRITER** integration, which gives us:
- **`strands-agents[writer]`** - AI agent framework with WRITER model support (includes `mcp` automatically)
- **`strands-agents-tools`** - Additional tools for file operations and web scraping

Let's start with the installations:

In [ ]:
%pip install "strands-agents[writer]" "strands-agents-tools"

## System prompt configuration

Here we define the system prompt that guides the Strands Agent’s behavior.
This prompt instructs the agent to:
- Act autonomously in reading, writing, and analyzing information.
- Use WRITER’s MCP-connected tools for file processing, editing, and knowledge graph interactions, etc.
- Provide clear, human-readable explanations and summaries of its work.


In [ ]:
system_prompt = """
You are a research and writing assistant built on the WRITER SDK and connected to the Model Control Protocol (MCP).

Your goals:
1. Help users research technical, creative, or analytical topics.
2. Produce clear, accurate, and well-structured text.
3. When using tools, explain what you’re doing and why, unless instructed otherwise.
4. Be concise and factual in tone.

Always prioritize correctness, clarity, and tool-assisted reasoning.
"""

## Creating the agent

This code creates a `StrandsAgent` class that:
- Connects to a local MCP server running on port 3000.
- Uses WRITER’s Palmyra X5 model for natural language processing and reasoning.
- Connects the model to the MCPClient, enabling access to available tools.
- Provides an interactive chat interface for research and development tasks.

In [ ]:
import sys
import logging
from mcp.client.streamable_http import streamablehttp_client
from mcp.shared.exceptions import McpError
from strands import Agent
from strands.models.writer import WriterModel
from strands.tools.mcp import MCPClient

class StrandsAgent:
    """Lightweight research agent using WRITER's MCP tools via Strands."""

    def __init__(self, system_prompt, server, base_url="http://localhost:3000"):
        self.sdk_client = MCPClient(lambda: streamablehttp_client(base_url))
        self.model = WriterModel(model_id="palmyra-x5", temperature=0.3, top_p=0.8)
        self.system_prompt = system_prompt
        self.server=server

    def chat_loop(self):
        """Start an interactive chat session with the MCP-connected agent."""
        print("\n🤖 Strands Agent (Palmyra X5) ready!")
        print("Ask about WRITER SDK, models, or Knowledge Graph tools.")
        print("Type 'quit' to exit.\n")

        with self.sdk_client:
            tools = self.sdk_client.list_tools_sync()
            agent = Agent(model=self.model, tools=tools, system_prompt=self.system_prompt)

            while True:
                user_input = input("🧑‍💻 You: ").strip()
                if user_input.lower() in {"quit", "exit"}:
                    print("👋 Goodbye!")
                    break
                if not user_input:
                    continue

                print("🔍 Researching...\n")
                try:
                    response = agent(user_input, stream=False)
                    print("🤖 Agent:", response, "\n")
                except McpError as e:
                    print(f"⚠️ MCP tool failed: {e}\n")
                except Exception as e:
                    print(f"⚠️ Unexpected error: {type(e).__name__}: {e}\n")
    def stop(self):
        """Stop the MCP server."""
        self.server.terminate()
        self.server.wait()
        print("🛑 WRITER MCP Server stopped.")


Initialize the **StrandsAgent** to connect the **WRITER MCP package** (Node SDK) with the **Strands Agents SDK**.
This lets your local agent use WRITER’s MCP tools through the Palmyra model interface.

**Try this query:**

> “List the applications I have access to.”

If you’ve created any [applications](https://dev.writer.com/home/applications#invoke-no-code-agents-via-the-api) in WRITER AI Studio, the agent will use the MCP tools to discover and list them — allowing you to then invoke those no-code agents directly via the API.

In [ ]:
# Initialize and run the Strands Research Agent
try:
    agent = StrandsAgent(system_prompt=system_prompt, server=server)
    agent.chat_loop()
finally:
    agent.stop()